### Importación de librerías

In [2]:
import cv2
import os
import mediapipe as mp

### Definición de variables

In [3]:
DATASET_NAME = "../../dataset/gestos_cara"
LABELS = {
    0: "0_Neutro",
    1: "1_Ojos_Cerrados",
    2: "2_Cabeza_Der",    # Girar cabeza a la derecha
    3: "3_Cabeza_Izq",    # Girar cabeza a la izquierda
    4: "4_Lengua_Fuera"   # Sacar lengua
}

### Lógica de guardado de imágenes

In [5]:
# Crear carpetas
if not os.path.exists(DATASET_NAME):
    os.makedirs(DATASET_NAME)

for key, nombre_carpeta in LABELS.items():
    path = os.path.join(DATASET_NAME, nombre_carpeta)
    if not os.path.exists(path):
        os.makedirs(path)

# Configuramos FaceMesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

face_mesh = mp_face_mesh.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True, # Importante para ver iris y detalles de boca
    min_detection_confidence=0.5
)

cap = cv2.VideoCapture(0)

# Contadores
contadores = {}
for key in LABELS:
    path = os.path.join(DATASET_NAME, LABELS[key])
    contadores[key] = len(os.listdir(path))

print(f"--- RECOLECTOR FACIAL ---")
print(f"Guardando en: {DATASET_NAME}")

while True:
    ret, frame = cap.read()
    if not ret: break
    
    frame = cv2.flip(frame, 1)
    frame_display = frame.copy()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = face_mesh.process(frame_rgb)
    
    cara_detectada = False
    if results.multi_face_landmarks:
        cara_detectada = True
        for face_landmarks in results.multi_face_landmarks:
            # Dibujamos contornos (ojos, labios, ovalo cara)
            mp_drawing.draw_landmarks(
                image=frame_display,
                landmark_list=face_landmarks,
                connections=mp_face_mesh.FACEMESH_CONTOURS,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())

    # === INTERFAZ ===
    fuente = cv2.FONT_HERSHEY_SIMPLEX
    
    # Resumen superior
    info = f"N:{contadores[0]} | Ojos:{contadores[1]} | Der:{contadores[2]} | Izq:{contadores[3]} | Leng:{contadores[4]}"
    cv2.putText(frame_display, info, (10, 30), fuente, 0.5, (0, 255, 255), 1)

    # Instrucciones inferiores
    lineas = [
        "0:Neutro | 1:Ojos Cerrados",
        "2:Cabeza Der | 3:Cabeza Izq",
        "4:Lengua Fuera | q:Salir"
    ]
    
    y0 = 420
    for i, linea in enumerate(lineas):
        y = y0 + i * 25
        (w, h), _ = cv2.getTextSize(linea, fuente, 0.6, 1)
        cv2.rectangle(frame_display, (10, y - h - 5), (10 + w + 10, y + 5), (0,0,0), -1)
        cv2.putText(frame_display, linea, (15, y), fuente, 0.6, (0, 255, 255), 1)

    if not cara_detectada:
        cv2.putText(frame_display, "CARA NO DETECTADA", (10, 200), fuente, 0.8, (0, 0, 255), 2)

    cv2.imshow('Recolector Facial', frame_display)
    
    # === GUARDADO ===
    k = cv2.waitKey(1)
    if k == ord('q'): break
    
    clase = -1
    if 48 <= k <= 52: # 0 al 4
        clase = k - 48
        
    if clase != -1:
        carpeta = LABELS[clase]
        idx = contadores[clase]
        path_img = os.path.join(DATASET_NAME, carpeta, f"{carpeta}_{idx}.jpg")
        cv2.imwrite(path_img, frame)
        print(f"Guardado: {path_img}")
        contadores[clase] += 1

cap.release()
cv2.destroyAllWindows()

--- RECOLECTOR FACIAL ---
Guardando en: ../../dataset/gestos_cara
Guardado: ../../dataset/gestos_cara\2_Cabeza_Der\2_Cabeza_Der_1.jpg
Guardado: ../../dataset/gestos_cara\2_Cabeza_Der\2_Cabeza_Der_2.jpg
